# Software Simulation

In [1]:
import numpy as np


N = 512
MOD = 7681
DELTA = 2  

def mulmod(a, b):
    return (a * b) % MOD

def addmod(a, b):
    res = a + b
    return res - MOD if res >= MOD else res

def submod(a, b):
    res = a - b
    return res + MOD if res < 0 else res

def modpow(x, e):
    res = 1
    while e:
        if e & 1:
            res = mulmod(res, x)
        x = mulmod(x, x)
        e >>= 1
    return res


def bit_reverse(x, logn):
    res = 0
    for i in range(logn):
        if x & (1 << i):
            res |= 1 << (logn - 1 - i)
    return res

def ntt(a, root=7146):
    logn = N.bit_length() - 1
    a = np.copy(a)
    
    for i in range(N):
        j = bit_reverse(i, logn)
        if i < j:
            a[i], a[j] = a[j], a[i]

    len_ = 2
    while len_ <= N:
        wlen = modpow(root, N // len_)
        for i in range(0, N, len_):
            w = 1
            for j in range(len_ // 2):
                u = a[i + j]
                v = mulmod(a[i + j + len_ // 2], w)
                a[i + j] = addmod(u, v)
                a[i + j + len_ // 2] = submod(u, v)
                w = mulmod(w, wlen)
        len_ <<= 1
    return a

def intt(a, root_inv=7480, n_inv=7666):
    a = ntt(a, root=root_inv)
    return [(mulmod(x, n_inv)) for x in a]


def ref_keygen(A, S, E):
    pk0 = np.zeros(N, dtype=np.uint32)
    # V = np.zeros(N, dtype=np.uint32)
    # W = np.zeros(N, dtype=np.uint32)
    # U = np.zeros(N, dtype=np.uint32)

    
    tA = A.copy()
    tS = S.copy()
    tA = ntt(tA)
    tS = ntt(tS)
    tA = np.array([mulmod(x, y) for x, y in zip(tA, tS)])
    tA = intt(tA)
    pk0 = np.array([addmod(tA[i], mulmod(E[i], DELTA)) for i in range(N)])

    
    # tA = A.copy()
    # tS = R.copy()
    # tA = ntt(tA)
    # tS = ntt(tS)
    # V = np.array([mulmod(x, y) for x, y in zip(tA, tS)])
    # V = intt(V)

    
    # tA = pk0.copy()
    # tS = R.copy()
    # tA = ntt(tA)
    # tS = ntt(tS)
    # W = np.array([mulmod(x, y) for x, y in zip(tA, tS)])
    # W = intt(W)
    # W = np.array([addmod(W[i], M[i]) for i in range(N)])

    
    # tA = V.copy()
    # tS = S.copy()
    # tA = ntt(tA)
    # tS = ntt(tS)
    # U = np.array([mulmod(x, y) for x, y in zip(tA, tS)])
    # U = intt(U)

    
    # output = np.array([submod(W[i], U[i]) & 1 for i in range(N)], dtype=np.uint32)
    # return output
    return pk0

In [2]:
# def string_to_bitarray(message):
#     bit_array = []
#     for c in message:
#         ascii_val = ord(c)  
#         bits = [(ascii_val >> i) & 1 for i in reversed(range(8))]
#         bit_array.extend(bits)
#     return np.array(bit_array, dtype=np.uint32)


In [3]:
# def bitarray_to_string(bit_array):
#     chars = []
#     for i in range(0, len(bit_array), 8):
#         byte = bit_array[i:i+8]
#         if len(byte) < 8:
#             break  
#         val = 0
#         for bit in byte:
#             val = (val << 1) | bit
#         chars.append(chr(val))
#     return ''.join(chars)


In [4]:
# A = np.array([i % MOD for i in range(N)], dtype=np.uint32)
# S = np.array([(i*3 + 1) % MOD for i in range(N)], dtype=np.uint32)
# E = np.ones(N, dtype=np.uint32)
# R = np.ones(N, dtype=np.uint32)
# M = np.array([i & 1 for i in range(N)], dtype=np.uint32)
# M=M*0
# M[0] = 1  

In [5]:
# message = "HELLO WORLD!"
# msg_bits = string_to_bitarray(message)
# bit_len = len(msg_bits)


# if bit_len > 512:
#     raise ValueError("Message too long! Max 64 characters.")

# M = np.zeros(512, dtype=np.uint32)
# M[:bit_len] = msg_bits


A = np.array([i % MOD for i in range(N)], dtype=np.uint32)
S = np.array([(i*3 + 1) % MOD for i in range(N)], dtype=np.uint32)
E = np.ones(N, dtype=np.uint32)
# R = np.ones(N, dtype=np.uint32)


In [6]:
pk = ref_keygen(A, S, E)

In [9]:
print(len(pk))

512


In [9]:
# print("Gold output:", gold)

In [10]:
print("Original message :", message)
print("Recovered message:", recovered_string)

Original message : HELLO WORLD!
Recovered message: HELLO WORLD!                                                    


# Hardware Simulation

In [13]:
from pynq import Overlay, allocate
import numpy as np



ModuleNotFoundError: No module named 'pynq'

In [ ]:

N = 512
MOD = 7681



In [ ]:

ol = Overlay("keygen.bit")
dma = ol.axi_dma_0  

in_buffer = allocate(shape=(N * 3,), dtype=np.uint32)  
out_buffer = allocate(shape=(N,), dtype=np.uint32)     

NameError: name 'Overlay' is not defined

In [ ]:
in_buffer[0::3] = A
in_buffer[1::3] = S
in_buffer[2::3] = E



NameError: name 'in_buffer' is not defined

In [18]:
dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer)

dma.sendchannel.wait()
dma.recvchannel.wait()



NameError: name 'dma' is not defined

In [19]:
print(out_buffer[:])

NameError: name 'out_buffer' is not defined

In [ ]:
# recovered_string_hw = bitarray_to_string(out_buffer)
# print("Recovered message from hardware:", recovered_string_hw)

NameError: name 'out_buffer' is not defined